
# Intelligent car counting with Intel® Geti™ SDK from Training to Deployment

## Smart Car Counting for Efficient Urban Management: Harnessing the Power of Intel Geti Platform, Intel Geti SDK, and OpenVINO Integration

In this comprehensive notebook, we will guide developers through the process of creating a smart car counting system for parking lots using the cutting-edge Intel Geti platform, its SDK, and seamless integration with OpenVINO. We will demonstrate how to train models, manipulate training features, and deploy the solution locally, all while reaping the numerous benefits of OpenVINO's accelerated inferencing capabilities.

| ![image.png](https://github.com/openvinotoolkit/geti-sdk/assets/76463150/5dc1d5a3-0ea3-42f0-831e-53489304d44e) | 
|:--:| 
| *Parking lot view - Smart Camera System* |

| ![image-7.png](https://github.com/openvinotoolkit/geti-sdk/assets/76463150/5357bd34-26a7-4f20-b4cb-3b56a9b4ee77) | 
|:--:| 
| *Intel® Geti™ Platform* |

Before running this notebook, please make sure you have the Intel Geti SDK installed in your local machine. If not, please follow [these instructions](https://github.com/openvinotoolkit/geti-sdk#installation).

In this notebook, we will use the SDK to create a project on the Intel Geti graphics platform, upload videos, download the displays locally, and run the inference by viewing the results on this same notebook.

## Requirements before to start

We need to download some data from a GitHub repo, so please, if it is the first time you run this notebook, be sure you run the next cell and restart the kernel after that to apply changes to your environment.

In [ ]:
%pip install -q "gitpython" "gdown"

# Step 1: Connect with your Intel® Geti™ Instance

We will connect to the platform first, using the server details from the .env file. We will also create a ProjectClient for the server. If you have doubts, please take a look of the previous work you need to do before to [connect](https://github.com/openvinotoolkit/geti-sdk#connecting-to-the-intel-geti-platform) the SDK and the platform. 

In [ ]:
# As usual we will connect to the platform first, using the server details from the .env file. We will also create a ProjectClient for the server
import os

from geti_sdk import Geti
from geti_sdk.rest_clients import ProjectClient
from geti_sdk.utils import get_server_details_from_env

geti_server_configuration = get_server_details_from_env(env_file_path=os.path.join("..", ".env"))

geti = Geti(server_config=geti_server_configuration)

project_client = ProjectClient(session=geti.session, workspace_id=geti.workspace_id)

# Step 2: Create a project and upload a video for further annotations

We will create a new project from scratch and will upload a video using the SDK for further annotations into the Intel Geti Platform. We will create an object detection project for person, bike and card detection. 

| ![image.png](https://github.com/openvinotoolkit/geti-sdk/assets/76463150/49dbe60f-4a7b-47dd-a42c-71ad28c93593) | 
|:--:| 
| *Video Frame from parking lot - car detection data* |

### Setting up the project client, video client and prediction client
For now, we will need two client objects: A `ProjectClient` to retrieve the project we want to upload to, and an `VideoClient` to be able to upload the video. We first set up the `ProjectClient`, since we will need to get the project we are interested in before we can initialize the another client.

In [ ]:
from geti_sdk.rest_clients import ImageClient, ProjectClient, VideoClient

project_client = ProjectClient(session=geti.session, workspace_id=geti.workspace_id)  # setting up the project client

projects = project_client.list_projects()  # listing the projects in the Intel Geti Instance

#### Project creation
For project creation we will need: 1. Project Name, 2. Project Type, and 3. Properties for each project task. See this notebook for an extensive [explanation](https://github.com/openvinotoolkit/geti-sdk/blob/main/notebooks/001_create_project.ipynb). For this use case we will create an detection task.

In [ ]:
# First set the project parameters. Feel free to experiment here!
PROJECT_NAME = "parking-lot"  # the project we want to create
PROJECT_TYPE = "rotated_detection"
LABELS = [["car"]]  # The label names for the task

In [ ]:
# Now, use the project client to create the project
project = project_client.get_or_create_project(project_name=PROJECT_NAME, project_type=PROJECT_TYPE, labels=LABELS)

# Step 3: Uploading images into the project

We can upload a video directly from file using the `image_client.upload_image()` method. Before uploading, we can get a list of all videos in the project, so that we can verify that the image was uploaded successfully. With the project name specified, we can retrieve the project details from the project client and use the returned `Project` object to set up an `image_client` and `prediction_client` for this project.

In [ ]:
project = project_client.get_project_by_name(PROJECT_NAME)

video_client = VideoClient(session=geti.session, workspace_id=geti.workspace_id, project=project)

image_client = ImageClient(session=geti.session, workspace_id=geti.workspace_id, project=project)

In [ ]:
# images in the project before uploading
images = image_client.get_all_images()
print(f"Project '{project.name}' contains {len(images)} images.")

Now, we will upload an example image folder from the SDK, so first we will download those images and annotations. 

In [ ]:
import os
from pathlib import Path

from git.repo import Repo

working_folder = os.getcwd()

current_directory = Path(os.path.join(Path.cwd(), "data", "103_geti_sdk_project"))

if not os.path.isfile(current_directory):
    repo = Repo.clone_from(
        url="https://github.com/paularamo/103_geti_sdk_project.git",
        to_path=current_directory,
    )

images_path = current_directory / "images"
annotations_path = current_directory / "annotations"

Upload the images using ```image_client.upload_folder``` function. Of course, you can replace the `images_path` with a path to one of your own videos as well.

In [ ]:
image_client.upload_folder(images_path);

Let's fetch the list of images again and see if it has changed

In [ ]:
# images in the project after uploading
images = image_client.get_all_images()
print(f"Project '{project.name}' contains {len(images)} images.")

# Step 4: Creating annotations
Once you upload new image or video data, you should open the Intel Geti GUI and create annotations for it. The screenshot below shows an example of the annotator page within Geti.

| ![image.png](https://github.com/openvinotoolkit/geti-sdk/assets/76463150/51ba8173-11a1-4cc5-8e24-2be0989afdf8) | 
|:--:| 
| *Annotations within the Intel® Geti™ Platform* |

Alternatively, if you have used the default 'parking-lot' image dataset that we provided, you can run the cell below to upload some pre-defined annotations for the images to the project. This saves you some time in annotating the images.

But before to upload the annotations, we need to disable the auto-training option from the Intel Geti server, for avoing auto training when the first  12 annotations have been submitted.

In [ ]:
# Disable auto-training option
from geti_sdk.rest_clients import ConfigurationClient

# initialize the client
cc = ConfigurationClient(workspace_id=geti.workspace_id, session=geti.session, project=project)
cc.set_project_auto_train(False)

In [ ]:
# uploading the annotations into the project
from geti_sdk.annotation_readers import GetiAnnotationReader
from geti_sdk.rest_clients import AnnotationClient

annotation_reader = GetiAnnotationReader(os.path.join(annotations_path))

annotation_client = AnnotationClient(
    workspace_id=geti.workspace_id,
    session=geti.session,
    project=project,
    annotation_reader=annotation_reader,
)

annotation_client.upload_annotations_for_all_media()

# Step 5: Training the model
Once sufficient annotations have been made, the project is ready for training. Due to the incremental learning mechanism within the Intel Geti platform, training will happen automatically and frequently. Whenever sufficient new annotations have been created, the platform will start a training round. 

In the next part of the notebook we will deploy the model that was trained, so that we can use it locally to generate predictions. However, before doing so we need to make sure that the project has a model trained. The cell below trigger the training process for all possible algorithms the Intel Geti Platform supports. Only one model is trained at a time for a project, so it will take some time but you could see the differences between architectures, and make the decision about which architecture is better for your use case



In [ ]:
from geti_sdk.rest_clients import TrainingClient

training_client = TrainingClient(session=geti.session, workspace_id=geti.workspace_id, project=project)
task = project.get_trainable_tasks()[0]

# list all the available algorithms for our task
available_algorithms = training_client.get_algorithms_for_task(task=task)

# start a training job for every algo available in the task
jobs = []
for algorithm in available_algorithms:
    jobs.append(training_client.train_task(algorithm=algorithm, task=task, enable_pot_optimization=False))

Training the models will take a while. We can monitor the training job progress and wait for them to complete using the code in the next cell. The progress monitoring will block further program execution, until the jobs have finished.

In [ ]:
training_client.monitor_jobs(jobs);

### Optimize/Quantize your models with OpenVINO

OpenVINO helps with the optimization and quantization process, there are multiple options to optimize and quantize the models. On of them is using the Post training Quantization process with POT. For triggering that option on the Intel Geti platform, you can fetch the trained model and then request an optimization job to optimize it with POT. The code in the next cell shows how this is done.

In [ ]:
from geti_sdk.rest_clients.model_client import ModelClient

model_client = ModelClient(session=geti.session, workspace_id=geti.workspace_id, project=project)

# fetch the trained models from the platform
models = []
for job in jobs:
    models.append(model_client.get_model_for_job(job))

# request model optimization with POT
optimization_jobs = []
for model in models:
    optimization_jobs.append(model_client.optimize_model(model))

# monitor the optimization job progress
training_client.monitor_jobs(optimization_jobs);

# Step 6: Download the model and save the deployment
When the model is ready you can download the deployment and run it locally or run the inference in the platform. In this example we will download the deployment and run it locally.

Once we are sure that the project has trained models for each task, we can create the deployment in the cell below.

In [ ]:
latest_trained_models = []
for algo in available_algorithms:
    model = model_client.get_latest_model_by_algo_name(algo.name)
    if model is not None:
        print(f"Retrieved latest trained model for algorithm {algo.name}")
        latest_trained_models.append(model)

# For each algorithm, grab the optimized models that are available. These will be used for local deployment
optimized_models = []
for model in latest_trained_models:
    optimized_openvino_models = [om for om in model.optimized_models if "OpenVINO" in om.name and not om.has_xai_head]
    print(
        f"Found {len(optimized_openvino_models)} optimized OpenVINO models for base model with architecture `{model.name}`"
    )
    optimized_models.extend(optimized_openvino_models)

print(f"A total of {len(optimized_models)} optimized OpenVINO models suitable for local deployment have been found")

In [ ]:
# Make sure we have a folder to store the deployments
deployment_folder_name = "deployments"
if not os.path.exists(deployment_folder_name):
    os.makedirs(deployment_folder_name)

# request a deployment for the optimized models
output_folder = f"{deployment_folder_name}/{PROJECT_NAME}"
print(f"Deploying models to folder {output_folder}")
for optimized_model in optimized_models:
    print(f"Creating deployment for model `{optimized_model.name}`")
    dst = f"{output_folder} {optimized_model.name}"
    geti.deploy_project(project_name=PROJECT_NAME, output_folder=dst, models=[optimized_model])
print(f"Model deployment complete. {len(optimized_models)} deployment folders have been generated.")

## Selecting the prefered deployment for running locally

When we create the deployment, the model data is saved to a temporary folder. We store the deployment for offline re-use later on by saving it: This will copy the model data from the temporary folder to the path we specify. If we want to run inference locally again, we can simply reload the deployment from the saved folder, without having to connect to the platform again.

In [ ]:
from geti_sdk.deployment import Deployment

model_of_interest = "MaskRCNN-ResNet50 OpenVINO FP16"

deployment_folder_model_selected = f"{working_folder}/{deployment_folder_name}/{PROJECT_NAME} {model_of_interest}"
deployment = Deployment.from_folder(deployment_folder_model_selected)

## Preparing the models for inference
Now that the `deployment` is created and the models are saved to the local disk, we can load the models into memory to prepare them for inference. There you can select the device for running the inference, in OpenVINO we have different options. You can setup `CPU`, `GPU`, `AUTO` for [auto plugin](https://docs.openvino.ai/latest/openvino_docs_OV_UG_supported_plugins_AUTO.html), and `MULTI` for runnig the model in [multiple devices](https://docs.openvino.ai/latest/openvino_docs_OV_UG_Running_on_multiple_devices.html).

In [ ]:
from openvino.runtime import Core

ie = Core()
devices = ie.available_devices

for device in devices:
    device_name = ie.get_property(device, "FULL_DEVICE_NAME")
    print(f"{device}: {device_name}")

In [ ]:
deployment.load_inference_models(device="CPU")

### Testing local inference on a single image
To make sure the deployment is working properly, let's quickly run inference on a single image from the project

In [ ]:
from geti_sdk.prediction_visualization.visualizer import Visualizer

image = images[0]
image_data = image.get_data(geti.session)

prediction = deployment.infer(image_data)

visualizer = Visualizer()
result = visualizer.draw(image_data, prediction)
visualizer.show_in_notebook(result)

# Step 7: Run the inference and ingest new data into the Platform

We will run the inference locally and send some detection frames to the Intel Geti Platform, in order to annotate those and retrain a new model.

What happens if something new comes in your production system? Different acquisition conditions, lighting, camera, backgrounds. You can connect your production system with Intel Geti Platform in a flexible way through the Intel Geti SDK.

Note: For this use case we will send images back to the Intel Geti Platform when the number of detections per frame will be higher than 1.

## Preparing payload function for sending back frames to the Platform
In `utils\upload.py` we will find an `Uploader` class to perform multithreaded uploading to the Geti platform. The main purpose of this is to avoid any delay in the video visualization in the notebook, while still being able to upload frames on-the-go. 

In [ ]:
from utils import Uploader

uploader = Uploader(num_worker_threads=2, image_client=image_client)

## Main function for running the inference with video files or USB camera

This main function create a video player object to manage video files or USB cameras. By default we play the video in 30 FPS, and every single frame will be analyzed by the model. It also runs the inference using the Intel Geti SDK and create a queue of frames to be sent back to the Intel Geti platform.

Essentially, the code has four main components:

1. `VideoPlayer(source=source, flip=flip, fps=30, skip_first_frames=skip_first_frames)`: Custom video player to fulfill FPS requirements, you can set a webcam or video file, target FPS and output size, flip the video horizontally or skip first N frames.


2. `prediction = deployment.infer(frame)`: This generates a prediction for the image or frame. The prediction contains a list of methods and variables, such as `annotations` which contains information about boundung boxes, labels, confidence and color.


3. `uploader.add_data(input_image)`: uploader is a class to help us to create a separate thread for sending images back to the platform. This class is using image_client for that purposes.


4. `Visualizer.draw(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR), prediction, show_results=False)`: this method helps us to have bounding boxes, labels and confidence over the actual frame for visualization purposes. 

In [ ]:
import collections
import time

import cv2
import numpy as np
from IPython import display
from utils import VideoPlayer

from geti_sdk.prediction_visualization.visualizer import Visualizer


def display_text_fnc(frame: np.ndarray, display_text: str, index: int):
    """
    Include a text on the analyzed frame

    :param frame: input frame
    :param display_text: text to add on the frame
    :param index: index line for adding text

    """
    # Configuration for displaying images with text.
    FONT_COLOR = (255, 0, 0)
    FONT_COLOR2 = (0, 0, 0)
    FONT_STYLE = cv2.FONT_HERSHEY_DUPLEX
    FONT_SIZE = 1
    TEXT_VERTICAL_INTERVAL = 25
    TEXT_LEFT_MARGIN = 800
    text_loc = (TEXT_LEFT_MARGIN, TEXT_VERTICAL_INTERVAL * (index + 1))
    text_loc2 = (TEXT_LEFT_MARGIN + 1, TEXT_VERTICAL_INTERVAL * (index + 1) + 1)
    cv2.putText(frame, display_text, text_loc2, FONT_STYLE, FONT_SIZE, FONT_COLOR2)
    cv2.putText(frame, display_text, text_loc, FONT_STYLE, FONT_SIZE, FONT_COLOR)


# Main processing function to run object detection.
def run_object_detection(source=0, flip=False, use_popup=False, skip_first_frames=0):
    visualizer = Visualizer()
    player = None
    fps = 0
    number_cars = 0
    try:
        # ===================1. Create a video player to play with target fps================
        player = VideoPlayer(source=source, flip=flip, fps=30, skip_first_frames=skip_first_frames)
        # Start capturing.
        player.start()
        if use_popup:
            title = "Press ESC to Exit"
            cv2.namedWindow(winname=title, flags=cv2.WINDOW_GUI_NORMAL | cv2.WINDOW_AUTOSIZE)

        processing_times = collections.deque()
        text_inference_template = "Detected Cars:{predictions:d}, {fps:.1f}FPS"

        while True:
            # Grab the frame.
            frame = player.next()
            if frame is None:
                print("Source ended")
                break
            # If the frame is larger than full HD, reduce size to improve the performance.
            scale = 1280 / max(frame.shape)
            height, width, channels = frame.shape
            # print(frame.shape)
            if scale <= 2:  # < 1:
                frame = cv2.resize(
                    src=frame,
                    dsize=None,
                    fx=1280 / width,
                    fy=720 / height,
                    interpolation=cv2.INTER_AREA,
                )
            input_image = frame.copy()

            # Measure processing time.

            start_time = time.time()

            # ==========================2. Using Geti SDK predictions========================

            # Get the results.
            prediction = deployment.infer(frame)
            stop_time = time.time()
            processing_times.append(stop_time - start_time)

            # ==========================3. Sending images back to the platform======================

            # if the prediction has more than one label send the image back to Intel Geti Platform
            if len(prediction.annotations) == 1:
                for annotation in prediction.annotations:
                    for label in annotation.labels:
                        if label.name == "car":
                            number_cars = 1
                        else:
                            number_cars = 0
            else:
                number_cars = len(prediction.annotations)
                uploader.add_data(input_image)
                print(f"queue = {uploader.queue_length}")

            # ================4. Creating output with bounding boxes, labels, and confidence========
            output = visualizer.draw(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), prediction)
            output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)

            # Use processing times from last 200 frames.
            if len(processing_times) > 200:
                processing_times.popleft()

            _, f_width = frame.shape[:2]
            # Mean processing time [ms].
            processing_time = np.mean(processing_times) * 100
            fps = 1000 / processing_time

            display_text = text_inference_template.format(predictions=number_cars, fps=fps)
            display_text_fnc(output, display_text, 1)

            # Use this workaround if there is flickering.
            if use_popup:
                cv2.imshow(winname=title, mat=output)
                key = cv2.waitKey(1)
                # escape = 27
                if key == 27:
                    break
            else:
                # Encode numpy array to jpg.
                _, encoded_img = cv2.imencode(ext=".jpg", img=output, params=[cv2.IMWRITE_JPEG_QUALITY, 100])
                # Create an IPython image.
                i = display.Image(data=encoded_img)
                # Display the image in this notebook.
                display.clear_output(wait=True)
                display.display(i)
    # ctrl-c
    except KeyboardInterrupt:
        print("Interrupted")
    # any different error
    except RuntimeError as e:
        print(e)
    finally:
        if player is not None:
            # Stop capturing.
            player.stop()
        if use_popup:
            cv2.destroyAllWindows()

# Step 8: Run the main function with video

Using the previous main function, we will run the inference in real time over this notebook and we will see the bounding boxes and the detection on a new video.

In [ ]:
import os

import gdown
from utils import PARKING_LOT_VIDEO_HASH

from geti_sdk.demos.data_helpers.download_helpers import validate_hash

url = "https://drive.google.com/uc?id=1Z2uQ63wkge8iw7tgGStDG6FvEyhi4dTS"
output = os.path.join("data", "parking.mp4")

gdown.download(url, output, quiet=False)
validate_hash(output, PARKING_LOT_VIDEO_HASH)

In [ ]:
# In this example, we run inference on the same video that we uploaded to the project. Of course `video_file`
# can be set to a different video for a more realistic scenario
from pathlib import Path

video_file = os.path.join(working_folder, output)
print(video_file)

run_object_detection(source=video_file, flip=False, use_popup=True)

This example showcases Intel® Geti™ SDK with a video file, but you can use live camera streams or so in a similar manner, just change the source argument in the previous function to make it equal the the camera source number.

# Step 9: Repeat Step 4, create/edit annotations in the platform
Once you upload new data to Geti, you should open the GUI and check, approve or edit the annotations. After accepting or editing a sufficient number of annotations, the platform will start a new round of model training. This training round takes your suggestions into account, in order to further improve the model.

When the model is ready you can download the deployment again and use it to obtain predictions, just like we did before. 


| ![image.png](https://github.com/openvinotoolkit/geti-sdk/assets/76463150/e48e98d1-37d6-4989-90b5-49ac106997dc)) | 
|:--:| 
| *Interactive annotation with the Intel® Geti™ Platform* |

Alternatively, if you have used the default 'person_car_bike' video that we provided, you can run the cell below to upload some pre-defined annotations for the video to the project. This saves you some time in annotating the frames.